In [1]:
import warnings

from sympy.solvers.diophantine.diophantine import length

warnings.filterwarnings("ignore", category=UserWarning)

In [2]:
# Установка необходимых библиотек:
!pip install scikit-learn
!pip install numpy
!pip install pandas
!pip install transformers[torch]
!pip install sentencepiece
!pip install datasets

/bin/bash: warning: setlocale: LC_ALL: cannot change locale (en_US.UTF-8)
/bin/bash: warning: setlocale: LC_ALL: cannot change locale (en_US.UTF-8)
/bin/bash: warning: setlocale: LC_ALL: cannot change locale (en_US.UTF-8)
/bin/bash: warning: setlocale: LC_ALL: cannot change locale (en_US.UTF-8)
/bin/bash: warning: setlocale: LC_ALL: cannot change locale (en_US.UTF-8)
/bin/bash: warning: setlocale: LC_ALL: cannot change locale (en_US.UTF-8)


In [3]:
import pandas as pd
import numpy as np
import random
from transformers import GPT2LMHeadModel, GPT2Tokenizer
from sklearn.model_selection import train_test_split
import torch

In [4]:
# Загружаем нужную модель
model_checkpoint = "sberbank-ai/rugpt3large_based_on_gpt2"
model = GPT2LMHeadModel.from_pretrained(model_checkpoint).eval()
tokenizer = GPT2Tokenizer.from_pretrained(model_checkpoint)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
if device.type == 'cuda':
    model = model.to(
        "cuda")  #перевод модели на GPU, можно обойтись и без этого, тогда генерация будет происходить медленнее

In [5]:
# Загрузка данных
BASE_DIR = "/home/alexandr/PycharmProjects/NLP_homework/homework3"
test_data = pd.read_csv(BASE_DIR + "/in_domain_dev.csv")
test_data = test_data[['sentence', 'acceptable']]
test_text = test_data['sentence']
test_labels = test_data['acceptable']

In [6]:
# Шаблон ввода: инструкция + вводимое предложение
def get_prompt_zero(sentence):
    prompt_template = f"""
    Оцени предложение на предмет правильности написания на русском языке, приемлимое или неприемлимое:
    Предложение: {sentence}
    Ваш выбор:"""
    return prompt_template

In [7]:
# Шаблон ввода: инструкция + вводимое предложение
def get_prompt_few(sentence):
    prompt_template = f"""
    Ты учитель русского языка и должен оценить следующее предложение на предмет языковой приемлемости:
    Предложение: {sentence}
    Варианты ответа:
    Приемлемое — предложение грамматически правильное. // Книги и журналы были на столе.
    Неприемлемое — предложение имеет нарушения грамматики, стиля, правил русского языка. // Иван войдя, он поздоровался.
    Ваш выбор:"""
    return prompt_template

In [8]:
# Функция для классификации предложений методом zero/few shot
def classify_sentence(prompt):
    inputs = tokenizer(prompt, return_tensors="pt").input_ids
    outputs = model.generate(inputs, max_length=min(512, len(prompt)), num_return_sequences=1)
    generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    answer = generated_text.split('Ваш выбор:')[-1].strip().lower()  # Извлекаем последний элемент ответа
    print(answer)
    if 'прием' in answer:
        return 'Приемлемое'
    else:
        return 'Неприемлемое'

In [9]:
example_sentence = "Иван войдя, он поздоровался."

classification_result_zero = classify_sentence(get_prompt_zero(example_sentence))
classification_result_few = classify_sentence(get_prompt_few(example_sentence))
print(f"Предложение: '{example_sentence}'\nКлассификация: {classification_result_zero}")
print(f"Предложение: '{example_sentence}'\nКлассификация: {classification_result_few}", end="\n\n")

example_sentence = "Книги и журналы были на столе."

classification_result_zero = classify_sentence(get_prompt_zero(example_sentence))
classification_result_few = classify_sentence(get_prompt_few(example_sentence))
print(f"Предложение: '{example_sentence}'\nКлассификация: {classification_result_zero}")
print(f"Предложение: '{example_sentence}'\nКлассификация: {classification_result_few}", end="\n\n")


здравствуйте, иван
приемлемое — предложение грамма
Предложение: 'Иван войдя, он поздоровался.'
Классификация: Неприемлемое
Предложение: 'Иван войдя, он поздоровался.'
Классификация: Приемлемое

книги и журналы были на столе.
    ваш
приемлем
Предложение: 'Книги и журналы были на столе.'
Классификация: Неприемлемое
Предложение: 'Книги и журналы были на столе.'
Классификация: Приемлемое



In [10]:
# Функция для классификации предложений методом zero/few shot
def classify_sentence_binary(prompt):
    inputs = tokenizer(prompt, return_tensors="pt").input_ids
    outputs = model.generate(inputs, max_length=min(512, len(prompt)), num_return_sequences=1)
    generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    answer = generated_text.split('Ваш выбор:')[-1].strip().lower()  # Извлекаем последний элемент ответа
    if 'прием' in answer:
        return 1
    else:
        return 0


In [11]:
# Применяем классификацию к примеру из датасета
pred_list_zero_shot = []
pred_list_few_shot = []

for sentence in test_text:
    classification_result_zero = classify_sentence_binary(get_prompt_zero(sentence))
    classification_result_few = classify_sentence_binary(get_prompt_few(sentence))
    pred_list_zero_shot.append(classification_result_zero)
    pred_list_few_shot.append(classification_result_few)

In [15]:
data_pred = test_data.copy()
data_pred['pred_zero'] = pred_list_zero_shot
data_pred['pred_few'] = pred_list_few_shot

data_pred

,sentence,acceptable,pred_zero,pred_few
0,Иван вчера не позвонил.,1,0,1
1,"У многих туристов, кто посещают Кемер весной, ...",0,0,0
2,Лесные запахи набегали волнами; в них смешалос...,1,0,1
3,Вчера президент имел неофициальную беседу с ан...,1,0,1
4,Коллега так и не признал вину за катастрофу пе...,1,0,1
...,...,...,...,...
978,Мысли отказываются остановиться на всяком пред...,0,0,1
979,"Не должно быть подозрений, что судью привлекаю...",0,0,0
980,"Хорошо, что он купил что-нибудь.",0,0,1
981,"Если бы я не потерял очков, не пришлось бы пок...",0,0,1


In [17]:
# Расчет метрик
def metric(fact, pred):
    if len(pred) != len(fact):
        return None
    tp, fp, fn, tn = 0, 0, 0, 0
    for x in range(len(pred)):
        if pred[x] == 1:
            if pred[x] == fact[x]:
                tp += 1
            else:
                fp += 1
        else:
            if pred[x] == fact[x]:
                tn += 1
            else:
                fn += 1
    accuracy = (tp + tn) / (tp + tn + fp + fn)
    recall = tp / (tp + fp)
    precision = fp / (tp + fn)
    f1 = (2 * precision * recall) / (precision + recall)
    return f1, accuracy

In [20]:
f1_zero = metric(
    test_data['acceptable'].tolist(),
    pred_list_zero_shot
)
f1_few = metric(
    test_data['acceptable'].tolist(),
    pred_list_few_shot,
)

print(f"F1 при использовании метода zero-shot: {f1_zero[0]:.3f}")
print(f"F1 при использовании метода few-shot: {f1_few[0]:.3f}")

F1 при использовании метода zero-shot: 0.003
F1 при использовании метода few-shot: 0.390
